# **Homework 1: COVID-19 Cases Prediction (Regression)**

# 全连接神经网络 使用的是**DNN** (Deep Neural Network，深度神经网络)/**MLP**多层感知机/**FCNN** 全连接神经网络。这个模型包含了三层隐藏层。
## 深度神经网络 (Deep Neural Network / DNN)：当一个全连接神经网络 (MLP) 包含多个隐藏层时，它就被称为深度神经网络。全连接神经网络 (Fully Connected Neural Network / FCNN) 或 多层感知机 (Multi-Layer Perceptron / MLP)：这是最基础的神经网络类型，每一层的神经元都与前一层的所有神经元相连。你的模型就是这种架构。
## 当一个MLP变得足够“深”（拥有足够的隐藏层），它就晋升为DNN了。EnhancedModel 既是一个全连接神经网络/多层感知机，也是一个深度神经网络。

# **新收获**
## **超参数**是在训练过程开始之前**手动设置的配置变量**。是自己设置的
## **参数**是模型在训练过程中**从数据中自动学习到的变量**。是模型自己算出来,学习到的

In [1]:
!wget -O covid_train.csv https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0
!wget -O covid_test.csv https://www.dropbox.com/s/zalbw42lu4nmhr2/covid.test.csv?dl=0

--2025-06-25 12:18:21--  https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/ewl0ff7lviu0s7f53jp9o/covid.train.csv?rlkey=pocojbo26thh2ncv0xkxfafiv&dl=0 [following]
--2025-06-25 12:18:22--  https://www.dropbox.com/scl/fi/ewl0ff7lviu0s7f53jp9o/covid.train.csv?rlkey=pocojbo26thh2ncv0xkxfafiv&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdf9f6e14e08fb1a6e5df2c0e95.dl.dropboxusercontent.com/cd/0/inline/CsT9_S_6w8sde5r7ck1j_e1Dwf2wkTdAz_hy4iZ6yfYefjNBfQZ4LE67qRtnHrA--NZoKAzTR3XWMq9aNZqKZdUz1KzALyGqzvp2IALxguR9IwmHMCmooZAlhLxzaq3htVdIZD1eZl5hNHtswe0xFil1/file# [following]
--2025-06-25 12:18:22--  https://ucdf9f6e14e08fb1a6e5df2c0e95.dl.dropboxuserc

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import pandas as pd
import numpy as np
import math
import os
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
print("Import Complete!")

Import Complete!


In [76]:
class COVID19Dataset(Dataset):
    def __init__(self, x, y=None,scaler = None):
        self.scaler = scaler
        if y is not None:
            self.scaler = StandardScaler()
            self.x = self.scaler.fit_transform(x)
            self.y = torch.FloatTensor(y)
        else:
            if scaler is None:
                raise ValueError("测试机需要提供预拟合的scaler")
            self.scaler =  scaler
            self.x = self.scaler.transform(x)
            self.y = None

    def __getitem__(self, idx):
        if self.y is None:
            return torch.FloatTensor(self.x[idx])
        return torch.FloatTensor(self.x[idx]), self.y[idx]

    def __len__(self):
        return len(self.x)

print("Class is defined!")

Class is defined!


In [44]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        # 使用SelectKBest选择最重要的10个特征
        selector = SelectKBest(f_regression, k=10)
        selector.fit(raw_x_train, y_train)
        feat_idx = selector.get_support(indices=True)
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid
print("func is defined!")

func is defined!


In [45]:
class EnhancedModel(nn.Module):
    def __init__(self, input_dim):
        super(EnhancedModel, self).__init__()
        
        self.net = nn.Sequential(
            # 全连接层,它接收 input_dim 个特征作为输入。它产生 64 个输出特征。这个层有 input_dim * 64 个权重和 64 个偏置。
            nn.Linear(input_dim, 64),
            # 批量归一化层,它对前一层（有 64 个特征）的输出在批次维度上进行归一化.它学习 64 个 gamma（缩放）参数和 64 个 beta（偏移）参数。
            nn.BatchNorm1d(64),
            # 激活函数,这允许负数输入也有小的梯度，有助于防止“ReLU死亡”问题。目的：引入非线性，使网络能够学习复杂的模式。
            nn.LeakyReLU(0.1),
            # Dropout 层,在训练期间，它会随机地将批次中每个样本的 30% (0.3) 的输入特征设置为零.目的：通过强制网络学习更鲁棒的特征并防止神经元之间的协同适应来防止过拟合。
            nn.Dropout(0.3),

            # nn.Linear(64, 32): 另一个全连接层。接收前一个块的 64 个特征。输出 32 个特征。参数：64 * 32 个权重和 32 个偏置。
            nn.Linear(64, 32),
            # 32 个特征的批量归一化。32 个 gamma 和 32 个 beta 参数。
            nn.BatchNorm1d(32),
            # Leaky ReLU 激活函数。
            nn.LeakyReLU(0.1),
            # Dropout 层，随机将 20% 的特征设置为零。目的：进一步正则化。
            nn.Dropout(0.2),

            # 全连接层。没有 Dropout 或 BatchNorm,接收 32 个特征。输出 16 个特征。参数：32 * 16 个权重和 16 个偏置。
            nn.Linear(32, 16),
            # Leaky ReLU 激活函数。
            nn.LeakyReLU(0.1),
            
            #  最后一个全连接层。输出1 个特征。这个单一输出表明该模型可能设计用于回归任务（预测一个连续值）或二分类（如果在此层之后应用 Sigmoid 激活）。
            # 参数：16 * 1 个权重和 1 个偏置。
            nn.Linear(16, 1)
        )
        
        # 初始化网络层的权重和偏置.
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                """
                m.weight: 指的是当前 nn.Linear 层的权重张量。

                mode='fan_in': 表示初始化的方差应该基于层的输入单元数量（fan-in）。

                nonlinearity='leaky_relu': 指定 Kaiming 初始化应该根据使用 Leaky ReLU 激活函数的层进行校准。
                """
                # 这种初始化策略常用于 ReLU 类激活函数，有助于防止训练过程中梯度消失或爆炸，尤其是在深度网络中。
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='leaky_relu')
                # 这行代码将 nn.Linear 层的偏置初始化为常量值 0.1。目的：一个小的正偏置有时有助于在训练初期激活 ReLU 类神经元。
                nn.init.constant_(m.bias, 0.1)

    def forward(self, x):
        return self.net(x).squeeze(1)
        # self.net(x): 这是关键所在。输入张量 x 被传入 nn.Sequential 容器 (self.net)，它会按顺序通过所有定义的层进行处理。
        # .squeeze(1): self.net(x) 的输出通常形状为 (batch_size, 1)。squeeze(1) 会移除索引为 1 的大小为 1 的维度。
        # 目的：这通常用于回归任务，因为你想要一个扁平的预测张量，而不是一个带有冗余的、大小为 1 的维度的张量。
print("Model is defined!")

Model is defined!


In [46]:
def trainer(train_loader, valid_loader, model, config, device):
    # 使用更鲁棒的损失函数
    criterion = nn.HuberLoss(delta=1.0)  # 对异常值更鲁棒
    
    # 优化器改进
    optimizer = torch.optim.AdamW(
        model.parameters(), 
        lr=config['learning_rate'],
        weight_decay=1e-4  # L2正则化
    )
    
    # 学习率调度
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='min',
        patience=20,
        factor=0.5,
        verbose=True
    )
    
    writer = SummaryWriter()
    os.makedirs('/kaggle/working/models', exist_ok=True)
    
    best_loss = float('inf')
    early_stop_count = 0
    
    for epoch in range(config['n_epochs']):
        model.train()
        train_loss = []
        train_pbar = tqdm(train_loader, desc=f'Epoch [{epoch+1}/{config["n_epochs"]}]')
        
        for x, y in train_pbar:
            x, y = x.to(device), y.to(device)

            # 清除优化器中所有累积的梯度。
            # 梯度在每次反向传播时都会累积，所以在新的批次开始时需要清零。
            optimizer.zero_grad()
            # 执行前向传播：将输入 x 传入模型，得到预测值 pred。
            pred = model(x)
            loss = criterion(pred, y)
            
            # 梯度裁剪：限制模型参数的梯度范数不超过 1.0。
            # 这有助于防止梯度爆炸问题，尤其是在训练深层网络或使用某些优化器时。
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # 执行反向传播：计算损失相对于模型所有可学习参数的梯度。
            # 这些梯度会存储在每个参数的 .grad 属性中。
            loss.backward()
            # 更新模型参数：根据计算出的梯度和优化器（例如 SGD, Adam 等）的规则来更新模型的权重和偏置。
            optimizer.step()
            
            train_loss.append(loss.item())
            train_pbar.set_postfix({'loss': loss.item()})
        
        # 验证阶段
        model.eval()
        valid_loss = []
        # 禁用梯度计算：
        # 在验证阶段，我们不需要计算梯度，因为我们不进行参数更新。
        # 禁用梯度可以减少内存消耗并加速计算。
        with torch.no_grad():
            for x, y in valid_loader:
                x, y = x.to(device), y.to(device)
                pred = model(x)
                loss = criterion(pred, y)
                valid_loss.append(loss.item())
        
        # 计算平均损失
        mean_train_loss = np.mean(train_loss)
        mean_valid_loss = np.mean(valid_loss)
        
        # 学习率调整
        # scheduler.step() 用于根据验证损失来调整学习率。
        # 例如，如果使用 ReduceLROnPlateau 调度器，当验证损失停止改善时，学习率可能会降低。
        scheduler.step(mean_valid_loss)
        
        # 记录和打印
        writer.add_scalars('Loss', {
            'train': mean_train_loss,
            'valid': mean_valid_loss
        }, epoch)
        
        print(f'Epoch {epoch+1}: Train Loss: {mean_train_loss:.4f} | Valid Loss: {mean_valid_loss:.4f}')
        
        # 早停机制
        # 如果当前 epoch 的验证损失低于历史最佳验证损失
        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss # 更新最佳损失
            # 保存当前模型的权重（state_dict），因为这是目前表现最好的模型。
            torch.save(model.state_dict(), config['save_path'])
            early_stop_count = 0 # 重置早停计数器
        else:
            early_stop_count += 1 # 否则，增加早停计数
            # 如果早停计数达到预设的阈值 (config['early_stop'])
            if early_stop_count >= config['early_stop']:
                print(f'\nEarly stopping at epoch {epoch+1}')
                break

print("trainer is defined!")

trainer is defined!


In [48]:
def save_pred(preds,file):
    with open(file,'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id','tested_positive'])
        for i,p in enumerate(preds):
            writer.writerow([i,p])

print("func is defined!")

func is defined!


In [77]:
def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    config = {
        'seed': 5201314,
        'select_all': False,  # 使用特征选择
        'valid_ratio': 0.2,
        'n_epochs': 1000,
        'batch_size': 512,
        'learning_rate': 1e-3,
        'early_stop': 100,
        'save_path': '/kaggle/working/models/best_model.ckpt'
    }

    # 设置随机种子
    torch.manual_seed(config['seed'])
    np.random.seed(config['seed'])
    
    # 加载数据
    train_data = pd.read_csv('/kaggle/working/covid_train.csv').values
    test_data = pd.read_csv('/kaggle/working/covid_test.csv').values
    
    # 数据分割
    train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])
    
    # 特征选择
    x_train, x_valid, x_test, y_train, y_valid = select_feat(
        train_data, valid_data, test_data, config['select_all'])
    
    print(f'Selected {x_train.shape[1]} features')
    
    # 创建数据集
    train_dataset = COVID19Dataset(x_train, y_train)
    fitted_scaler = train_dataset.scaler
    valid_dataset = COVID19Dataset(x_valid, y_valid,scaler=fitted_scaler)
    test_dataset = COVID19Dataset(x_test,scaler=fitted_scaler)
    
    # 数据加载器
    train_loader = DataLoader(
        train_dataset, 
        batch_size=config['batch_size'], 
        shuffle=True, 
        pin_memory=True,
        num_workers=2
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=config['batch_size'],
        shuffle=False,
        pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=config['batch_size'],
        shuffle=False,
        pin_memory=True
    )
    
    # 初始化模型
    model = EnhancedModel(input_dim=x_train.shape[1]).to(device)
    
    # 训练模型
    trainer(train_loader, valid_loader, model, config, device)
    
    # 预测和保存结果
    model.load_state_dict(torch.load(config['save_path']))
    preds = predict(test_loader, model, device)
    save_pred(preds, 'pred.csv')

# 这个结构提供了一个“入口点”，让你可以在一个文件中同时实现两种功能：
#作为可执行的脚本（直接运行）。
#作为可导入的模块（供其他文件调用）。
if __name__ == '__main__':
    main()

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Selected 10 features


Epoch [1/1000]: 100%|██████████| 5/5 [00:00<00:00, 34.40it/s, loss=15.9]


Epoch 1: Train Loss: 15.8823 | Valid Loss: 16.7533


Epoch [2/1000]: 100%|██████████| 5/5 [00:00<00:00, 36.72it/s, loss=14.9]


...

Epoch [324/1000]: 100%|██████████| 5/5 [00:00<00:00, 39.55it/s, loss=1.5]


Epoch 324: Train Loss: 1.5084 | Valid Loss: 0.5889


Epoch [325/1000]: 100%|██████████| 5/5 [00:00<00:00, 40.20it/s, loss=1.52]


Epoch 325: Train Loss: 1.5546 | Valid Loss: 0.5978

Early stopping at epoch 325


100%|██████████| 2/2 [00:00<00:00, 193.29it/s]
